In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./activity.csv',sep=',')
df = df.dropna()
df=df.drop(["company_type","target"],axis='columns')
df.dropna(subset=["interests","fields"], inplace=True)

#관심분야 더미변수화
df = df.join(df["interests"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#activity_benefit 더미변수
df = df.join(df["activity_benefit"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#fields 더미변수
df['fields'] = df['fields'].replace('기타', 'fields_기타', regex=True)

df = df.join(df["fields"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#우대역량 더미변수화
df["prefer"] = df["prefer"].str.replace(", 기타", "")                                                      #',기타'가 규칙없이 붙어있는경우가 많아서 삭제
df['prefer'] = df['prefer'].replace('기타', '우대역량_기타')                                                 #'기타'열이 관심분야에도 중복됨으로 교체 
df = df.join(df["prefer"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#모집인원
df["모집 인원"]=df["recruit"].str.extract(r'(\d+)')     #숫자만 남기기
df=df.replace({"모집 인원":'000'},'999')                #000은 세자리수를 의미함으로 999명으로 변경
df=df.replace({"모집 인원":'00'},'99')                  #00은 두자리수를 의미함으로 99명으로 변경
df=df.replace({"모집 인원":'0'},'9')                    #0은 힌자리수를 의미함으로 9명으로 변경
df=df.replace({"모집 인원":np.NaN},0)                   #NaN(미정)값 0으로 변경
df=df.astype({"모집 인원":'int'})                   
student_bins=[0,1,21,101,50000]                        #범주화  0:인원미정/1-20(명):소/21-100(명):중/101-50000(명):대
student_scale=["인원미정","소","중","대"]               
df = pd.concat([df, pd.cut(df["모집 인원"], bins=student_bins, right=False, labels=student_scale).rename("모집인원규모")], axis=1) #분포범주화



#열삭제
df=df.drop(["app_period","activity_period"],axis='columns')

#열위치조정


#데이터저장
df=df.to_csv("activity_process.csv")
